## Metrics Search Requests
This notebook explains how to retrieve citation data and other metrics for a given bibcode(s).

The base URL for metrics queries is `https://api.adsabs.harvard.edu/v1/metrics`. 

All queries to the API must include an HTTP header that specifies your API access token, e.g.

`curl -H 'Authorization: Bearer <token>' ...`

In the examples below, {token_header} is used to denote `-H 'Authorization: Bearer <token>'`, where `<token>` is your personal API access token as described in the top-level [README](https://github.com/adsabs/adsabs-dev-api/blob/master/README.md).

Note: the API examples in this notebook generally assume you're using `curl` statements from the shell command line--you'll see `%%bash` at the top of a cell if its contents should be run in the shell instead of in Python. Guidelines for translating bash `curl` statements into Python can be found in the [converting curl to python notebook](Converting_curl_to_python.ipynb). If you prefer, you can use the unofficial Python package instead; see the top-level [README](https://github.com/adsabs/adsabs-dev-api/blob/master/README.md) for more information.

### Sending a metrics request
All metrics requests use the default HTTP method POST. POST requests require a payload to be specified using the `-d` flag. For metrics, this payload will be JSON formatted, with the key `bibcodes` and a value set to a comma-separated list of the bibcodes to retrieve metrics for. For example:

`curl {token_header} https://api.adsabs.harvard.edu/v1/metrics -X POST -d '{"bibcodes": ["bibcode1","bibcode2"]}'`

You can request a specific set of metrics by adding the `types` parameter in the JSON payload. For example:

`curl {token_header} -H "Content-Type: application/json" https://api.adsabs.harvard.edu/v1/metrics -X POST -d '{"bibcodes": ["bibcode1","bibcode2"], "types":["basic"]}'`

This will return just the basic statistics for a set of bibcodes. The following values are allowed for the `types` list:

  * 'basic': publication and usage stats (all papers, and just refereed papers)
  * 'citations': citation stats
  * 'indicators': indicators, like the h-index, g-index, m-index etc etc
  * 'histograms': publication, citation, reads and downloads histograms
  * 'timeseries': time series for a set of indicators
  
To only return a specific histogram, add a `histograms` parameter to the JSON payload, specifying that histogram. For example:

`curl {token_header} -H "Content-Type: application/json" https://api.adsabs.harvard.edu/v1/metrics -X POST -d '{"bibcodes":["bibcode1","bibcode2"], "types":["histograms"], "histograms":["publications"]}'`

to get just the publications histogram. 

### Response from a metrics request
The service returns JSON with attributes determined by the contents of the `types` list (no such list means that everything gets returned). It always contains an attribute `skipped bibcodes`, which is a list of bibcodes for which no metrics data could be found. The following JSON attributes are returned by each type:

  * 'basic': 'basic stats' and 'basic stats refereed'
  * 'citations': 'citation stats' and 'citation stats refereed'
  * 'indicators': 'indicators' and 'indicators refereed'
  * 'histograms': 'histograms'
  * 'timeseries': 'time series'

Each of these attributes contains specific data elements. The following sections shows what each type contains (each bullet point is an attribute name, or JSON key).

#### 'basic stats' and 'basic stats refereed'
  * number of papers
  * normalized paper count
  * median number of reads
  * average number of reads
  * total number of reads
  * normalized number of reads
  * median number of downloads
  * average number of downloads
  * total number of downloads
  * normalized number of downloads
  
#### 'citation stats' and 'citation stats refereed'
  * number of citing papers
  * median number of citations
  * average number of citations
  * total number of citations
  * normalized number of citations
  * median number of refereed citations
  * average number of refereed citations
  * total number of refereed citations
  * normalized number of refereed citations
  * number of self-citations

#### 'indicators'
  * i10
  * i100
  * h
  * m
  * g
  * read10
  * tori

#### 'histograms'
For all histograms, the actual histogram values are stored as follows:
```
    {
	    "histogram name": {year: value, year: value, ...}
	}
```
where values can be either integers or floats, depending on the type of histograms. The histogram name is an attribute of the histogram type. For example, the `publications` type has 4 separate histograms: `all publications`, `refereed publications`, `all publications normalized` and `refereed publications normalized`, which are organized as follows in the JSON output:
```
    {
	    "histograms":  {
			"publications": {
				"all publications": {},
				"refereed publications": {},
				"all publications normalized": {},
				"refereed publications normalized": {},
			
			}
		}
	}
```
The following lists all histogram types and the names of the actual histograms stored in them:

  * 'publications': 'all publications', 'refereed publications', 'all publications normalized', 'refereed publications normalized'
  * 'reads': 'all reads', 'refereed reads', 'all reads normalized', 'refereed reads normalized'
  * 'downloads': 'all downloads', 'refereed downloads', 'all downloads normalized', 'refereed downloads normalized'
  * 'citations': 'refereed to refereed', 'nonrefereed to refereed', 'refereed to nonrefereed', 'nonrefereed to nonrefereed', 'refereed to refereed normalized', 'nonrefereed to refereed normalized', 'refereed to nonrefereed normalized', 'nonrefereed to nonrefereed normalized'

Note that 'refereed reads' means: 'number of reads for the refereed publications in the set'.

#### 'timeseries'
The `timeseries` attribute contains time series data for: `h`, `g`, `i10`, `i100`, `read10` and `tori`. The output is structured as
```
    {
		"time series": {
			"h": { values },
			"g": { values },
			"i10": { values },
			"i100": { values },
			"read10": { values },
			"tori": { values },
		
		}
	}
```
where the 'values' are organized in the same way as for the histograms: 
```
    {
	    "indicator name": {year: value, year: value, ...}
	}
```

### Examples

In [1]:
%%bash
token="your-token-here"
# get all metrics for a single bibcode
curl -H "Authorization: Bearer $token" -H "Content-Type: application/json" \
    https://api.adsabs.harvard.edu/v1/metrics -X POST -d '{"bibcodes": ["2003ApJS..148..175S"]}' | python -m json.tool 

{
    "basic stats": {
        "average number of downloads": 8134.0,
        "average number of reads": 19644.0,
        "median number of downloads": 8134.0,
        "median number of reads": 19644.0,
        "normalized paper count": 0.058823529411764705,
        "number of papers": 1,
        "recent number of downloads": 144,
        "recent number of reads": 446,
        "total number of downloads": 8134,
        "total number of reads": 19644
    },
    "basic stats refereed": {
        "average number of downloads": 8134.0,
        "average number of reads": 19644.0,
        "median number of downloads": 8134.0,
        "median number of reads": 19644.0,
        "normalized paper count": 0.058823529411764705,
        "number of papers": 1,
        "recent number of downloads": 144,
        "recent number of reads": 446,
        "total number of downloads": 8134,
        "total number of reads": 19644
    },
    "citation stats": {
        "average number of citations": 8730.0,


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6007  100  5970  100    37   5970     37  0:00:01 --:--:--  0:00:01 24518


In [2]:
%%bash
token="your-token-here"
# get basic metrics for multiple bibcodes
curl -H "Authorization: Bearer $token" -H "Content-Type: application/json" \
    https://api.adsabs.harvard.edu/v1/metrics \
    -X POST -d '{"bibcodes": ["2003ApJS..148..175S", "2007ApJS..170..377S"], "types":["basic"]}' | python -m json.tool

{
    "basic stats": {
        "average number of downloads": 7555.0,
        "average number of reads": 18185.5,
        "median number of downloads": 7555.0,
        "median number of reads": 18185.5,
        "normalized paper count": 0.10427807486631016,
        "number of papers": 2,
        "recent number of downloads": 240,
        "recent number of reads": 759,
        "total number of downloads": 15110,
        "total number of reads": 36371
    },
    "basic stats refereed": {
        "average number of downloads": 7555.0,
        "average number of reads": 18185.5,
        "median number of downloads": 7555.0,
        "median number of reads": 18185.5,
        "normalized paper count": 0.10427807486631016,
        "number of papers": 2,
        "recent number of downloads": 240,
        "recent number of reads": 759,
        "total number of downloads": 15110,
        "total number of reads": 36371
    },
    "skipped bibcodes": []
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   851  100   772  100    79    772     79  0:00:01 --:--:--  0:00:01  3976


In [3]:
%%bash
token="your-token-here"
# get data for a single histogram for multiple bibcodes
curl -H "Authorization: Bearer $token" -H "Content-Type: application/json" \
    https://api.adsabs.harvard.edu/v1/metrics \
    -X POST \
    -d '{"bibcodes": ["2003ApJS..148..175S", "2007ApJS..170..377S"], "types":["histograms"], "histograms":["citations"]}' \
    | python -m json.tool

{
    "histograms": {
        "citations": {
            "nonrefereed to nonrefereed": {
                "2001": 0,
                "2002": 0,
                "2003": 0,
                "2004": 0,
                "2005": 0,
                "2006": 0,
                "2007": 0,
                "2008": 0,
                "2009": 0,
                "2010": 0,
                "2011": 0,
                "2012": 0,
                "2013": 0,
                "2014": 0,
                "2015": 0,
                "2016": 0,
                "2017": 0,
                "2018": 0
            },
            "nonrefereed to nonrefereed normalized": {
                "2001": 0,
                "2002": 0,
                "2003": 0,
                "2004": 0,
                "2005": 0,
                "2006": 0,
                "2007": 0,
                "2008": 0,
                "2009": 0,
                "2010": 0,
                "2011": 0,
                "2012": 0,
                "2013": 0,
     

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2650  100  2538  100   112   2538    112  0:00:01 --:--:--  0:00:01  5567
